# 1. 学习率调整基本情况
```
# 基类/父类
class _LRScheduler():
    def __init__(self, optimizer, last_epoch=-1)
    ···
    def step(self):
    ···
    def get_lr(self):  # 在基类/父类中是一个虚函数，提醒子类中需要实现这个方法
        raise NotImplementedError
    ···
```
**主要属性：**
- optimizer：关联的优化器，学习率是在优化器中发挥作用的，LRScheduler 会去修改优化器中的学习率。
- last_epoch：记录 epoch 数，通常不会去设置它。
- base_lrs：是一个 list，记录初始学习率，通过访问优化器管理的参数组而获取，通常不会去设置它。

**主要方法：**
- step()：更新下一个 epoch 的学习率；**注意：学习率的更新 scheduler_lr.step() 是以 epoch 为周期的，而不是 iteration；训练时进行梯度更新 optimizer.step() 是以 iteration 为周期的。**
- get_lr()：在基类/父类中是一个虚函数，计算下一个 epoch 的学习率


# 2. PyTorch 提供的 6 种学习率调整策略

## 2.1 StepLR()

`scheduler_lr = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma=0.1, last_epoch=-1)`

功能：等间隔调整学习率

主要参数：
- step_size：调整间隔数，间隔多少个 epoch 去调整学习率
- gamma：调整系数，调整时当前学习率乘以的系数（0.1、0.5等）

调整方式：lr = lr * gamma

```
class StepLR(_LRScheduler):
    def __init__(self, ):
    ···
    def get_lr(self):  # 在子类中 overwrite get_lr() 方法，写入实际代码，用来计算下一个 epoch 的学习率
    ···
```
使用 StepLR(_LRScheduler) 子类时，scheduler_lr.step() 方法直接从基类/父类中调用

## 2.2 MultiStepLR()
```
milestones = [50, 75, 150]
scheduler_lr = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones, gamma=0.1)
```
功能：指定间隔调整学习率

主要参数：
- milestones：是一个 list，指定调整间隔，指定间隔多少个 epoch 去调整学习率
- gamma：调整系数，调整时当前学习率乘以的系数（0.1、0.5等）

调整方式：lr = lr * gamma

## 2.3 ExponentialLR()

`scheduler_lr = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1, last_epoch=-1)`

功能：按指数衰减调整学习率

主要参数：
- gamma：指数的底，通常为接近 1 的数（0.95 等）

调整方式：lr = lr * (gamma ** epoch)

## 2.4 CosineAnnealingLR()
```
T_max = 50
scheduler_lr = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)
```
功能：按照余弦周期调整学习率

主要参数：
- T_max：学习率的下降周期，为多少个 epoch，下降之后再上升，整个调整周期（回复到原来学习率的值）为 2*T_max 个 epoch
- eta_min：学习率下降的下限，通常设为 0

## 2.5 ReduceLROnPlateau()

`scheduler_lr = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=10, verbose=False, cooldown=0, min_lr=0, eps=1e-08)`

功能：监控某个指标（loss_value/accuray，标量），当这个指标不再变化时则调整学习率

主要参数：
- mode：当监控的指标是 loss_value 时，使用 min 模式；当监控的指标是 accuray 时，使用 max 模式
- factor：调整系数，相当于 gamma
- patience：“耐心”，最多接受所监控指标连续多少个 epoch 不变化才去调整学习率
- cooldown：“冷却时间”，每次调整完学习率之后，停止监控指标多少个 epoch
- verbose：是否打印日志，默认为 False
- min_lr：学习率下限
- eps：学习率衰减最小值

使用这种学习率调整策略时，需要在学习率更新代码中加入所监控指标：`scheduler_lr.step(loss_value)` 或 `scheduler_lr.step(accuray)`

## 2.6 LambdaLR()
```
weights_1 = torch.randn((6, 3, 5, 5))
weights_2 = torch.ones((5, 5))

optimizer = optim.SGD([{'params': [weights_1]}, {'params': [weights_2]}], lr=lr_init)

lambda1 = lambda epoch: 0.1 ** (epoch // 20)
lambda2 = lambda epoch: 0.95 ** epoch

lr_lambda=[lambda1, lambda2]
scheduler_lr = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1)
```
功能：自定义调整策略，不同的参数组对应各自的调整策略，weights_1：lambda1，weights_2：lambda2

主要参数：
- lr_lambda：一个 function 或包含多个 function 的 list

**总结：前四种为有序调整，第五种为自适应调整，第六种为自定义调整**